<a href="https://colab.research.google.com/github/shawn-maybush/3D-Kinematics-Analytics/blob/main/quaternions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import sympy as sp
from math import pi
import numpy as np
from IPython.display import display, Math
import altair as alt
import pandas as pd
from sympy.utilities.iterables import numbered_symbols

In [3]:
L1 = 30.0
L2 = 90.0
L3 = 110.0

def dualq_mul(*args):


    if len(args) < 2:
        raise Exception("Need at least 2 dual quaternions to multiply")

    result = args[0]
    for i in range(1, len(args)):



        dq_i = args[i]

        r1 = result[0]
        d1 = result[1]
        r2 = dq_i[0]
        d2 = dq_i[1]

        real_part = r1.mul(r2)
        dual_part = r1.mul(d2).add(d1.mul(r2))
        result = [real_part, dual_part]

    return result



<h2>Forward Kinematics</h2>

In [112]:
alpha_1, alpha_2, alpha_3 = sp.symbols('alpha_1 alpha_2 alpha_3', real=True)

In [113]:
r_c = sp.Quaternion.from_axis_angle([0, 0, 1], alpha_1)
t_c = sp.Quaternion(0, 0, 0, 0)
d_c = 0.5 * t_c.mul(r_c)
Q_c = [r_c, d_c]
display(Math(sp.latex(Q_c)))

<IPython.core.display.Math object>

In [114]:
r_f = sp.Quaternion.from_axis_angle([0, 1, 0], alpha_2)
t_f = sp.Quaternion(0, L1, 0, 0)
d_f = 0.5 * t_f.mul(r_f)
Q_f = [r_f, d_f]
display(Math(sp.latex(Q_f)))

<IPython.core.display.Math object>

In [115]:
r_t = sp.Quaternion.from_axis_angle([0, 1, 0], alpha_3)
t_t = sp.Quaternion(0, L2, 0, 0)
d_t = 0.5 * t_t.mul(r_t)
Q_t = [r_t, d_t]
display(Math(sp.latex(Q_t)))

<IPython.core.display.Math object>

In [116]:
r_e = sp.Quaternion.from_axis_angle([1, 1, 1], 0)
t_e = sp.Quaternion(0, L3, 0, 0)
d_e = 0.5 * t_e.mul(r_e)
Q_e = [r_e, d_e]
display(Math(sp.latex(Q_e)))

<IPython.core.display.Math object>

In [117]:
foot_pose_dq = dualq_mul(Q_c, Q_f, Q_t, Q_e)
foot_pose_dq = sp.trigsimp(foot_pose_dq)
display(Math(sp.latex(foot_pose_dq)))

<IPython.core.display.Math object>

In [118]:
from sympy.codegen.ast import real, float80
foot_pose_dq_real = foot_pose_dq[0]
foot_pose_dq_dual = foot_pose_dq[1]
foot_position = 2 * foot_pose_dq_dual.mul(foot_pose_dq_real.conjugate())
c_foot_position = foot_position.to_Matrix(vector_only=True).tolist()
c_x = c_foot_position[0][0]
c_y = c_foot_position[1][0]
c_z = c_foot_position[2][0]

In [119]:
c_x = sp.simplify(c_x)
sp.ccode(c_x, assign_to="x", type_aliases={real: float80})

'x = (90.0L*cosl(alpha_2) + 110.0L*cosl(alpha_2 + alpha_3) + 29.9999999999999964473L)*cosl(alpha_1);'

In [120]:
c_y = sp.simplify(c_y)
sp.ccode(c_y, assign_to="y", type_aliases={real: float80})

'y = (180.0L*cosl(alpha_2) + 220.0L*cosl(alpha_2 + alpha_3) + 60.0L)*sinl((1.0L/2.0L)*alpha_1)*cosl((1.0L/2.0L)*alpha_1);'

In [121]:
c_z = sp.simplify(c_z)
sp.ccode(c_z, assign_to="z", type_aliases={real: float80})

'z = -90.0L*sinl(alpha_2) + 7.10542735760100185871e-15L*sinl(alpha_2 - alpha_3) - 110.0L*sinl(alpha_2 + alpha_3);'

In [122]:
values = {alpha_1: 0, alpha_2: 0, alpha_3: pi/2}
foot_position_num = foot_position.evalf(subs=values, n=5)
foot_position_num = foot_position_num.to_Matrix(vector_only=True).tolist()
foot_orientation_num = foot_pose_dq[0].evalf(subs=values, n=5)
foot_orientation_num = foot_orientation_num.to_Matrix(vector_only=True).tolist()

x = round(foot_position_num[0][0], 2)
y = round(foot_position_num[1][0], 2)
z = round(foot_position_num[2][0], 2)
print(f"Foot Position:\nx:{x}\ny:{y}\nz:{z}\n")
print(f"Foot Orientation:\nx:{round(foot_orientation_num[0][0], 2)}\ny:{round(foot_orientation_num[1][0], 2)}\nz:{round(foot_orientation_num[2][0], 2)}\n")

Foot Position:
x:120.00
y:0
z:-110.00

Foot Orientation:
x:0
y:0.71
z:0



<h2>Inverse Kinematics</h2>

In [123]:
theta_1, theta_2, theta_3 = sp.symbols('theta_1 theta_2 theta_3', real=True)
ax_1, ax_2, ax_3 = sp.symbols('ax_1 ax_2 ax_3', real=True)
ay_1, ay_2, ay_3 = sp.symbols('ay_1 ay_2 ay_3', real=True)
az_1, az_2, az_3 = sp.symbols('az_1 az_2 az_3', real=True)

In [223]:
def dualq_exp(dq):

    phi = sp.symbols('phi', real=True)
    h_w, h_v = dq[0].scalar_part(), dq[0].to_Matrix(vector_only=True)
    d_w, d_v = dq[1].scalar_part(), dq[1].to_Matrix(vector_only=True)
    h_v_norm = sp.sqrt(h_v[0]**2 + h_v[1]**2 + h_v[2]**2)


    gamma = (h_v.T * d_v)[0]
    c = sp.cos(phi)
    s = sp.sin(phi)

    kappa_r = sp.series(s/phi, phi, 0, 5).removeO()
    kappa_d = sp.series(gamma*(c - kappa_r/phi**2), phi, 0, 5).removeO()
    a_r_v = kappa_r * h_v
    a_r = sp.Quaternion(c, *a_r_v)

    a_dv_1 = kappa_r * d_v
    a_dv_2 = kappa_d * h_v

    a_dv = a_dv_1 + a_dv_2

    a_dw = -(kappa_r * phi)
    a_d = sp.Quaternion(a_dw, *a_dv)

    e_w_1 = sp.exp(h_w)
    e_w_2 = d_w * sp.exp(h_w)

    real_part = e_w_1 * a_r
    dual_part = e_w_2 * a_r + e_w_1 * a_d

    real_part = real_part.subs(phi, h_v_norm)
    dual_part = dual_part.subs(phi, h_v_norm)

    return [real_part, dual_part]


In [224]:
r_ci = sp.Quaternion.from_axis_angle([ax_1, ay_1, az_1], theta_1)
t_ci = sp.Quaternion(0, 0, 0, 0)
d_ci = 0.5 * t_ci.mul(r_ci)
Q_c_IK = [r_ci, d_ci]
Q_c_IK_exp = dualq_exp(Q_c_IK)
display(Math(sp.latex(Q_c_IK_exp)))

<IPython.core.display.Math object>

In [225]:
r_fi = sp.Quaternion.from_axis_angle([ax_2, ay_2, az_2], theta_2)
t_fi = sp.Quaternion(0, L1, 0, 0)
d_fi = 0.5 * t_fi.mul(r_fi)
Q_f_IK = [r_fi,d_fi]
Q_f_IK_exp = dualq_exp(Q_f_IK)
display(Math(sp.latex(Q_f_IK_exp)))

<IPython.core.display.Math object>

In [226]:
r_ti = sp.Quaternion.from_axis_angle([ax_3, ay_3, az_3], theta_3)
t_ti = sp.Quaternion(0, L2, 0, 0)
d_ti = 0.5 * t_ti.mul(r_ti)
Q_t_IK = [r_ti, d_ti]
Q_t_IK_exp = dualq_exp(Q_t_IK)
display(Math(sp.latex(Q_t_IK_exp)))

<IPython.core.display.Math object>

In [227]:
r_ei = sp.Quaternion.from_axis_angle([0.0, 1.0, 0.0], 0.0)
t_ei = sp.Quaternion(0.0, L3, 0, 0.0)
d_ei = 0.5 * t_ei.mul(r_ei)
Q_ei = [r_ei, d_ei]
display(Math(sp.latex(Q_ei)))

<IPython.core.display.Math object>

In [228]:
foot_pose_dqi = dualq_mul(Q_c_IK, Q_f_IK, Q_t_IK, Q_ei)
display(Math(sp.latex(foot_pose_dqi)))
foot_position_IK = 2 * foot_pose_dqi[1].mul(foot_pose_dqi[0].conjugate())
foot_position_IK = foot_position_IK.to_Matrix(vector_only=True)
foot_orientation_IK = foot_pose_dqi[0].to_Matrix(vector_only=True)

<IPython.core.display.Math object>

In [229]:
#Joint 1

# gradient WRT theta_1
r_dot_theta_1 = Q_c_IK_exp[0].diff(theta_1)
r_dot_ax_1 = Q_c_IK_exp[0].diff(ax_1)
r_dot_ay_1 = Q_c_IK_exp[0].diff(ay_1)
r_dot_az_1 = Q_c_IK_exp[0].diff(az_1)

# Angular velocity (ω)
w_theta_1 = 2 * r_dot_theta_1.mul(Q_c_IK[0].conjugate())
w_theta_1 = w_theta_1.to_Matrix(vector_only=True)

w_ax_1 = 2 * r_dot_ax_1.mul(Q_c_IK[0].conjugate())
w_ax_1 = w_ax_1.to_Matrix(vector_only=True)

w_ay_1 = 2 * r_dot_ay_1.mul(Q_c_IK[0].conjugate())
w_ay_1 = w_ay_1.to_Matrix(vector_only=True)

w_az_1 = 2 * r_dot_az_1.mul(Q_c_IK[0].conjugate())
w_az_1 = w_az_1.to_Matrix(vector_only=True)

# Linear velocity (v)
v_theta_1 = (2 * Q_c_IK_exp[1].mul(Q_c_IK[0].conjugate())).diff(theta_1)
v_theta_1 = v_theta_1.to_Matrix(vector_only=True)

v_ax_1 = (2 * Q_c_IK_exp[1].mul(Q_c_IK[0].conjugate())).diff(ax_1)
v_ax_1 = v_ax_1.to_Matrix(vector_only=True)

v_ay_1 = (2 * Q_c_IK_exp[1].mul(Q_c_IK[0].conjugate())).diff(ay_1)
v_ay_1 = v_ay_1.to_Matrix(vector_only=True)

v_az_1 = (2 * Q_c_IK_exp[1].mul(Q_c_IK[0].conjugate())).diff(az_1)
v_az_1 = v_az_1.to_Matrix(vector_only=True)



In [230]:
#Joint 2

# gradient WRT theta_2
r_dot_theta_2 = Q_f_IK_exp[0].diff(theta_2)
r_dot_ax_2 = Q_f_IK_exp[0].diff(ax_2)
r_dot_ay_2 = Q_f_IK_exp[0].diff(ay_2)
r_dot_az_2 = Q_f_IK_exp[0].diff(az_2)

# Angular velocity (ω)
w_theta_2 = 2 * r_dot_theta_2.mul(Q_f_IK[0].conjugate())
w_theta_2 = w_theta_2.to_Matrix(vector_only=True)

w_ax_2 = 2 * r_dot_ax_2.mul(Q_f_IK[0].conjugate())
w_ax_2 = w_ax_2.to_Matrix(vector_only=True)

w_ay_2 = 2 * r_dot_ay_2.mul(Q_f_IK[0].conjugate())
w_ay_2 = w_ay_2.to_Matrix(vector_only=True)

w_az_2 = 2 * r_dot_az_2.mul(Q_f_IK[0].conjugate())
w_az_2 = w_az_2.to_Matrix(vector_only=True)

# Linear velocity (v)
v_theta_2 = (2 * Q_f_IK_exp[1].mul(Q_f_IK[0].conjugate())).diff(theta_2)
v_theta_2 = v_theta_2.to_Matrix(vector_only=True)

v_ax_2 = (2 * Q_f_IK_exp[1].mul(Q_f_IK[0].conjugate())).diff(ax_2)
v_ax_2 = v_ax_2.to_Matrix(vector_only=True)

v_ay_2 = (2 * Q_f_IK_exp[1].mul(Q_f_IK[0].conjugate())).diff(ay_2)
v_ay_2 = v_ay_2.to_Matrix(vector_only=True)

v_az_2 = (2 * Q_f_IK_exp[1].mul(Q_f_IK[0].conjugate())).diff(az_2)
v_az_2 = v_az_2.to_Matrix(vector_only=True)


In [231]:
#Joint 3

# Gradient WRT theta_3
r_dot_theta_3 = Q_t_IK_exp[0].diff(theta_3)
r_dot_ax_3 = Q_t_IK_exp[0].diff(ax_3)
r_dot_ay_3 = Q_t_IK_exp[0].diff(ay_3)
r_dot_az_3 = Q_t_IK_exp[0].diff(az_3)

# Angular velocity (ω)
w_theta_3 = 2 * r_dot_theta_3.mul(Q_t_IK[0].conjugate())
w_theta_3 = w_theta_3.to_Matrix(vector_only=True)

w_ax_3 = 2 * r_dot_ax_3.mul(Q_t_IK[0].conjugate())
w_ax_3 = w_ax_3.to_Matrix(vector_only=True)

w_ay_3 = 2 * r_dot_ay_3.mul(Q_t_IK[0].conjugate())
w_ay_3 = w_ay_3.to_Matrix(vector_only=True)

w_az_3 = 2 * r_dot_az_3.mul(Q_t_IK[0].conjugate())
w_az_3 = w_az_3.to_Matrix(vector_only=True)

# Linear Velocity (v)
v_theta_3 = (2 * Q_t_IK_exp[1].mul(Q_t_IK[0].conjugate())).diff(theta_3)
v_theta_3 = v_theta_3.to_Matrix(vector_only=True)

v_ax_3 = (2 * Q_t_IK_exp[1].mul(Q_t_IK[0].conjugate())).diff(ax_3)
v_ax_3 = v_ax_3.to_Matrix(vector_only=True)

v_ay_3 = (2 * Q_t_IK_exp[1].mul(Q_t_IK[0].conjugate())).diff(ay_3)
v_ay_3 = v_ay_3.to_Matrix(vector_only=True)

v_az_3 = (2 * Q_t_IK_exp[1].mul(Q_t_IK[0].conjugate())).diff(az_3)
v_az_3 = v_az_3.to_Matrix(vector_only=True)


In [232]:
joint_1_w = sp.Matrix.hstack(w_theta_1, w_ax_1, w_ay_1, w_az_1)
joint_1_v = sp.Matrix.hstack(v_theta_1, v_ax_1, v_ay_1, v_az_1)
joint_1 = sp.Matrix.vstack(joint_1_v, joint_1_w)

joint_2_w = sp.Matrix.hstack(w_theta_2, w_ax_2, w_ay_2, w_az_2)
joint_2_v = sp.Matrix.hstack(v_theta_2, v_ax_2, v_ay_2, v_az_2)
joint_2 = sp.Matrix.vstack(joint_2_v, joint_2_w)

joint_3_w = sp.Matrix.hstack(w_theta_3, w_ax_3, w_ay_3, w_az_3)
joint_3_v = sp.Matrix.hstack(v_theta_3, v_ax_3, v_ay_3, v_az_3)
joint_3 = sp.Matrix.vstack(joint_3_v, joint_3_w)

In [233]:
J = sp.Matrix.hstack(joint_1, joint_2, joint_3)

In [234]:
variables = [theta_1, ax_1, ay_1, az_1, theta_2, ax_2, ay_2, az_2, theta_3, ax_3, ay_3, az_3]
J_Lambda = sp.lambdify(variables, J, modules=["numpy"], cse=True, docstring_limit=0)

In [235]:
foot_position_IK_lambda = sp.lambdify(variables, foot_position_IK, modules=["numpy"], cse=True, docstring_limit=0)
foot_orientation_IK_lambda = sp.lambdify(variables, foot_orientation_IK, modules=["numpy"], cse=True, docstring_limit=0)

In [236]:
test_angles = np.array([pi/2, 0, 0, 1, 0.00001, 0, 1, 0, 0.000001, 0, 1, 0])
print(f'Foot Position: {foot_position_IK_lambda(*test_angles).flatten()}')
print(f'Foot Orientation: {foot_orientation_IK_lambda(*test_angles).flatten()}')

Foot Position: [ 2.31739061e-14  2.30000000e+02 -2.11000000e-03]
Foot Orientation: [-3.88908730e-06  3.88908730e-06  7.07106781e-01]


In [276]:

target_ef = np.array([-0.0000004, 0.0000004, 0.7, 0.0000001, 230, 0.0000001,], dtype=np.float64).flatten()

# Set convergence criteria
max_error = 0.5
max_iterations = 2000
delta_t = 0.01
damping_factor = 0.01


#Current Angles are in the zero position (Joint 1 0.00001 ange to avoid NaN)
current_angles = np.array([0.000001, 0, 0, 1, 0.000001, 0, 1, 0, 0.000001, 0, 1, 0], dtype=np.float64)

analytics_df = pd.DataFrame()

for i in range(max_iterations):

  analytics = {'delta_t':delta_t*i}


  print("______________________________________________________________________________________________________________________________________________________________")
  print(f"Iteration: {i} Started")


  # 1. Substitute current joint angles into the desired pose
  current_foot_position = foot_position_IK_lambda(*current_angles).flatten()
  current_foot_orientation = foot_orientation_IK_lambda(*current_angles).flatten()
  current_foot_pose = np.concatenate((current_foot_orientation, current_foot_position))




  print(f"current_foot_pose: {current_foot_pose}")
  print(f"target_foot_pose: {target_ef}")

  # 2. Calculate error between current and desired pose
  error = (target_ef - current_foot_pose)
  print(f"error = {error}")

  error_norm = np.linalg.norm(error)

  # 3. Check for convergence
  if error_norm < max_error:
        print("Converged!")
        break
  print(f"error.norm() = {error_norm}")

  analytics['error_norm'] = error_norm


  # 4. Substitute current joint angles into the Jacobian
  J_num = J_Lambda(*current_angles)
  print(f"J_num = {J_num}")

  condition_number = np.linalg.cond(J_num)
  print(f"condition_number = {condition_number}")
  analytics['condition_number'] = condition_number


  # 5. Calculate joint velocities (handle singularities, perhaps with damped least squares)
  # W⁻¹J^T (JWJ^T + λ²I)⁻¹
  # J_num = 3 x 12 matrix
  rank = np.linalg.matrix_rank(J_num)
  print(f"rank = {rank}")
  if rank < 6 or condition_number > 10:
    J_damped = np.linalg.inv(J_num.T @ J_num + damping_factor**2 * np.eye(12)) @ J_num.T
    q_dot = J_damped @ error
    print("Dampened")

  else:
    q_dot = np.linalg.pinv(J_num) @ error
    print("Not Dampened")
  print(f"q_dot = {q_dot}")

  delta_theta = q_dot * delta_t
  print(f"delta_theta = {delta_theta}")




  # 6. Update joint angles (integrate velocities)
  print(f"previous_angles = {current_angles}")
  current_angles = current_angles + delta_theta
  if current_angles[0] > pi/2:
    current_angles[0] = pi/2
  if current_angles[4] > pi/2:
    current_angles[4] = pi/2
  if current_angles[8] > pi/2:
    current_angles[8] = pi/2

  if current_angles[0] < -pi/2:
    current_angles[0] = -pi/2
  if current_angles[4] < -pi/2:
    current_angles[4] = -pi/2
  if current_angles[8] < -pi/2:
    current_angles[8] = -pi/2


  analytics['Qx'] = current_foot_pose[0]
  analytics['Qy'] = current_foot_pose[1]
  analytics['Qz'] = current_foot_pose[2]
  analytics['Tx'] = current_foot_pose[3]
  analytics['Ty'] = current_foot_pose[4]
  analytics['Tz'] = current_foot_pose[5]


  analytics['delta_theta_1'] = delta_theta[0]
  analytics['delta_ax_1'] = delta_theta[1]
  analytics['delta_ay_1'] = delta_theta[2]
  analytics['delta_az_1'] = delta_theta[3]
  analytics['delta_theta_2'] = delta_theta[4]
  analytics['delta_ax_2'] = delta_theta[5]
  analytics['delta_ay_2'] = delta_theta[6]
  analytics['delta_az_2'] = delta_theta[7]
  analytics['delta_theta_3'] = delta_theta[8]
  analytics['delta_ax_3'] = delta_theta[9]
  analytics['delta_ay_3'] = delta_theta[10]
  analytics['delta_az_3'] = delta_theta[11]

  analytics['theta_1'] = current_angles[0]
  analytics['ax_1'] = current_angles[1]
  analytics['ay_1'] = current_angles[2]
  analytics['az_1'] = current_angles[3]
  analytics['theta_2'] = current_angles[4]
  analytics['ax_2'] = current_angles[5]
  analytics['ay_2'] = current_angles[6]
  analytics['az_2'] = current_angles[7]
  analytics['theta_3'] = current_angles[8]
  analytics['ax_3'] = current_angles[9]
  analytics['ay_3'] = current_angles[10]
  analytics['az_3'] = current_angles[11]

  analytics_df = pd.concat([analytics_df, pd.DataFrame([analytics])], ignore_index=True)

  print(f"new_joint_1 = {current_angles[0]}")
  print(f"new_joint_2 = {current_angles[4]}")
  print(f"new_joint_3 = {current_angles[8]}")


  print(f"current_angles = {current_angles}")
  print(f"Iteration: {i} ended")
  print("______________________________________________________________________________________________________________________________________________________________")


# If we didn't converge, let the user know
if i == max_iterations - 1:
    print("Failed to converge.")

# Print the final joint angles (whether converged or not)
print("Final joint angles:")
print(f"Joint 1: {current_angles[0]}")
print(f"Joint 2: {current_angles[4]}")
print(f"Joint 3: {current_angles[8]}")

Streaming output truncated to the last 5000 lines.
rank = 6
Dampened
q_dot = [ 4.19296785e+00 -1.86591469e-01 -2.65046581e-02  2.47487436e-02
 -9.63446009e-02  2.91929573e-03 -3.34967403e-03 -1.18669284e-02
 -2.38021205e-02  4.99879082e-04  4.72926528e-04  3.61949624e-02]
delta_theta = [ 4.19296785e-02 -1.86591469e-03 -2.65046581e-04  2.47487436e-04
 -9.63446009e-04  2.91929573e-05 -3.34967403e-05 -1.18669284e-04
 -2.38021205e-04  4.99879082e-06  4.72926528e-06  3.61949624e-04]
previous_angles = [ 1.57079633  0.24938661  0.1809565   2.07396612  0.09836469  1.88981348
  1.55643568  0.02556453  0.023086    1.13755505  0.93293586 -0.02790029]
new_joint_1 = 1.5707963267948966
new_joint_2 = 0.09740124513860177
new_joint_3 = 0.022847983359117665
current_angles = [ 1.57079633  0.2475207   0.18069145  2.07421361  0.09740125  1.88984267
  1.55640219  0.02544586  0.02284798  1.13756005  0.93294059 -0.02753834]
Iteration: 1776 ended
________________________________________________________________

In [277]:
joint_angles = ['theta_1', 'theta_2', 'theta_3']
end_effector_position = ['Tx', 'Ty', 'Tz']
joint_angles_df = analytics_df[joint_angles + ['delta_t']]
end_effector_position_df = analytics_df[end_effector_position + ['delta_t']]


error_chart = alt.Chart(analytics_df).mark_line().encode(
    x='delta_t:Q',
    y='error_norm:Q',
    tooltip=['delta_t', 'error_norm']
).properties(
    title='Error Norm Over Time'
).interactive()

condition_chart = alt.Chart(analytics_df).mark_line().encode(
    x='delta_t:Q',
    y='condition_number:Q',
    tooltip=['delta_t', 'condition_number']
).properties(
    title='Condition Number Over Time'
).interactive()


joint_angle_chart = alt.Chart(joint_angles_df).transform_fold(
    joint_angles,
    as_=['joint', 'angle']
).mark_line().encode(
    x='delta_t:Q',
    y=alt.Y('angle:Q', title='Angle (radians)'),
    color='joint:N',
    tooltip=['delta_t:Q', 'angle:Q']
).properties(
    title='Joint Angles Over Time'
).interactive()


position_chart = alt.Chart(end_effector_position_df).transform_fold(
    end_effector_position,
    as_=['axis', 'value']
).mark_line().encode(
    x='delta_t:Q',
    y=alt.Y('value:Q', title='Position'),
    color='axis:N',
    tooltip=['delta_t:Q', 'value:Q']
).properties(
    title='End-Effector Position Over Time'
).interactive()

dashboard = alt.hconcat(error_chart,condition_chart, joint_angle_chart, position_chart)

dashboard.title = f'Target: x: {target_ef[3]:.2f}  y: {target_ef[4]:.2f}  z: {target_ef[5]:.2f}'

dashboard.display()


alt.HConcatChart(...)